In [137]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

pd.set_option('display.max_columns', 70)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 80)                      # 출력할 열의 너비

In [138]:
# 필요한 데이터 불러오기
data1 = pd.read_csv('data_for_service_1.csv')
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열_10개만.csv')

In [139]:
# 여행 키워드 단어와 가중치를 입력하시오!
# 단 키워드 단어는 
# ['돈','쇼핑','시장','싸다, 저렴하다','비싸다','먹다','음식, 식사','상품','구매',
# '가격','호텔','박물관','전시관','관광지','유적지','미술관','역사','유익하다','문화','궁전','전쟁','활동','놀이공원','게임',
# '친구','커플','재미','아쿠아리움','엔터테인먼트','아이','가족','경기','힐링','공원','휴식','풍경','자연','전망대','산책',
# '분수','여유','나들이','공연','콘서트','이벤트','영화','예술','미술','강좌','요리','공예','체험'] 중에서 선택해야한다.

# 대략 5개의 키워드가 입력되었을 경우(실제로는 이렇게 안 되겠지만, 5개 상위 키워드로 뽑아서 진행할 것임)

keyword1 = input('첫 번째 여행 키워드는? : ')
weight1 = input('위 키워드의 가중치는? : ')
keyword2 = input('두 번째 여행 키워드는? : ')
weight2 = input('위 키워드의 가중치는? : ')
keyword3 = input('세 번째 여행 키워드는? : ')
weight3 = input('위 키워드의 가중치는? : ')
keyword4 = input('네 번째 여행 키워드는? : ')
weight4 = input('위 키워드의 가중치는? : ')
keyword5 = input('5 번째 여행 키워드는? : ')
weight5 = input('위 키워드의 가중치는? : ')

keyword = [keyword1, keyword2, keyword3, keyword4, keyword5]
weight = [int(weight1), int(weight2), int(weight3), int(weight4), int(weight5)]

user_data = pd.DataFrame({'keyword':keyword, 'weight':weight})
user_data['rate'] = user_data['weight']/(sum(user_data['weight']))
user_data = user_data.sort_values('rate', ascending = False)
user_data = user_data.reset_index()
print(user_data)

첫 번째 여행 키워드는? : 전시관
위 키워드의 가중치는? : 10
두 번째 여행 키워드는? : 예술
위 키워드의 가중치는? : 15
세 번째 여행 키워드는? : 미술관
위 키워드의 가중치는? : 20
네 번째 여행 키워드는? : 궁전
위 키워드의 가중치는? : 10
5 번째 여행 키워드는? : 자연
위 키워드의 가중치는? : 5
   index keyword  weight      rate
0      2     미술관      20  0.333333
1      1      예술      15  0.250000
2      0     전시관      10  0.166667
3      3      궁전      10  0.166667
4      4      자연       5  0.083333


In [140]:
data1

,id,place,tag_list,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']",74,4.540541,"음식, 식사",쇼핑,아이,구매,구매
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']",203,4.206897,상품,비싸다,"음식, 식사","싸다, 저렴하다","싸다, 저렴하다"
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']",66,4.181818,"음식, 식사",박물관,상품,쇼핑,쇼핑
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']",75,4.386667,"음식, 식사",쇼핑,시장,가격,가격
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']",634,4.113565,"음식, 식사",시장,먹다,"싸다, 저렴하다","싸다, 저렴하다"
...,...,...,...,...,...,...,...,...,...,...
1123,2297,디보 빌리지,['쇼핑'],1,1.000000,쇼핑,Nan,Nan,Nan,Nan
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']",36,4.666667,"음식, 식사",관광지,재미,전시관,전시관
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']",22,4.363636,공원,재미,엔터테인먼트,공연,공연
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']",3,4.666667,예술,공연,"음식, 식사",체험,체험


In [141]:
# 가상 장소 확정 짓기 (키워드 별로 뽑기)
keyword_order = list(user_data['keyword'])
print(keyword_order)

# 키워드가 들어간 장소는 다 뽑기(쉬운 모델링을 위해 상위 3개만 고려함....)
firstword_table = data1[data1['tag_list'].str.contains(keyword_order[0])]
secondword_table = data1[data1['tag_list'].str.contains(keyword_order[1])]
thirdword_table = data1[data1['tag_list'].str.contains(keyword_order[2])]

entire_table = pd.concat([firstword_table,secondword_table,thirdword_table])
entire_table = entire_table.drop_duplicates()

if len(entire_table) >= 100:
    next_table = firstword_table
else : next_table = entire_table
    
# 한 번 걸러진,next_table에서 우선순위 단어 고려해보기
A1 = next_table[next_table['first_tag'] == keyword_order[0]]
A2 = next_table[next_table['second_tag'] == keyword_order[0]]
A3 = next_table[next_table['third_tag'] == keyword_order[0]]
important_A = pd.concat([A1,A2,A3])
important_A = important_A.drop_duplicates()

if len(next_table) >= 50:
    semi_table = A1
elif len(next_table) >= 20:
    semi_table = important_A
else : semi_table = next_table

B1 = semi_table[semi_table['first_tag'] == keyword_order[1]]
B2 = semi_table[semi_table['second_tag'] == keyword_order[1]]
B3 = semi_table[semi_table['third_tag'] == keyword_order[1]]
B4 = semi_table[semi_table['fourth_tag'] == keyword_order[1]]
B5 = semi_table[semi_table['fifth_tag'] == keyword_order[1]]

important_B = pd.concat([B1,B2,B3,B4,B5])
important_B = important_B.drop_duplicates()

if len(semi_table) >=30:
    if len(important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]) >= 1:
        final_table = important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]
    else : final_table = important_B[(important_B['first_tag'] == keyword_order[0]) | (important_B['second_tag'] == keyword_order[1])]
elif len(semi_table) >=10:
    final_table = important_B[important_B['first_tag'] == keyword_order[0]]
else : final_table = semi_table

# 최종 spot 선정
if len(final_table) >=2:
    end_table = final_table.sort_values('count', ascending=False).iloc[:3,:] 
else : end_table = final_table

# 최종 spot 장소( 3개 이하)
print(end_table)

['미술관', '예술', '전시관', '궁전', '자연']
      id           place                                           tag_list  \
235  591  아라리오뮤지엄 인 스페이스  ['미술관', '미술', '예술', '음식, 식사', '박물관', '궁전', '전시관']   
264  620       아모레퍼시픽미술관  ['미술관', '미술', '예술', '박물관', '전시관', '쇼핑', '음식, 식사']   
92    94          인사아트센터                   ['미술관', '미술', '예술', '문화', '전시관']   

     count  avg_score first_tag second_tag third_tag fourth_tag fifth_tag  
235     30   4.633333       미술관         미술        예술     음식, 식사    음식, 식사  
264      7   4.571429       미술관         미술        예술        박물관       박물관  
92       2   4.500000       미술관         미술        예술         문화        문화  


In [142]:
# end_table이 완성이 되었다면, 여기서 1~3 개 정도의 spot을 가상으로 잡아, 유사도 높은 장소를 각가 2개씩 뽑아오기
# 원래는 이 과정에서 여행 타입 별로 (비율적으로 분할해야한다...)
virtual_place_name = []
virtual_place_id = []
virtual_place_index = end_table.index.to_list()

for i in range(len(end_table)):
    vname = end_table.iloc[i,1]
    vid = end_table.iloc[i,0]
    virtual_place_name.append(vname)
    virtual_place_id.append(vid)
print(virtual_place_name)
print(virtual_place_id)
print(virtual_place_index)

['아라리오뮤지엄 인 스페이스', '아모레퍼시픽미술관', '인사아트센터']
[591, 620, 94]
[235, 264, 92]


In [149]:
# 각 장소 별 2군데 추가로 유사도로 선정

recommend_id = []
recommend_name = []
recommend_index = []

for i in range(3):
    recom = sim_order_table[sim_order_table['0']== virtual_place_name[i]]
    spot1 = recom.iloc[0,1]
    spot2 = recom.iloc[0,2]
    recommend_index.append(virtual_place_index[i])
    recommend_index.append(spot1)
    recommend_index.append(spot2)
    
for j in recommend_index:
    find_name = sim_order_table.iloc[j,0]
    recommend_name.append(find_name)

for k in recommend_index:
    find_id = data1.iloc[k,0]
    recommend_id.append(find_id)
    
print(recommend_index)
print(recommend_name)
print(recommend_id)


[235, 227, 264, 264, 82, 235, 92, 236, 812]
['아라리오뮤지엄 인 스페이스', '국립현대미술관 서울관', '아모레퍼시픽미술관', '아모레퍼시픽미술관', '인사동 마루', '아라리오뮤지엄 인 스페이스', '인사아트센터', '서울 시립 미술관', '주한중국문화원']
[591, 583, 620, 620, 83, 591, 94, 592, 1636]


In [151]:
# 최종 추천지 반환하기!!! (중복 당연히 제거)

final_recommend_id = []
final_recommend_name = []

for a in recommend_id:
    if data1[data1['id'] == a].iloc[0,3]>=3:
        final_recommend_id.append(a)
final_recommend_id = list(set(final_recommend_id))

for k in final_recommend_id:
    final_name = data1[data1['id']==k].iloc[0,1]
    final_recommend_name.append(final_name)

    
print("추천 결과는??") 
Recommendation = pd.DataFrame({'id':final_recommend_id, 'name' : final_recommend_name})
Recommendation

추천 결과는??


,id,name
0,583,국립현대미술관 서울관
1,620,아모레퍼시픽미술관
2,591,아라리오뮤지엄 인 스페이스
3,592,서울 시립 미술관
4,83,인사동 마루
